In [1]:
import os
import pdfplumber
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import markdown
from bs4 import BeautifulSoup
import re
import unicodedata
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fati1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Document Parsing

### Data Loading 

File Path here

In [2]:
file_path = './problemStatement.md'

Get the file extension and parse it accordingly

In [3]:
# get file extension
def get_file_extension(file_path):
    return os.path.splitext(file_path)[-1].lower()


In [4]:
# use the appropriate text parser based on file extension (pdf, markdown, txt)
def parse_file(file_path):
    file_extension = get_file_extension(file_path)
    
    if file_extension == '.pdf':
        with pdfplumber.open(file_path) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() + '\n'
            return text
    elif file_extension == '.md' or file_extension == '.markdown':
        with open(file_path, "r", encoding="utf-8") as file:
            html = markdown.markdown(file.read())
        return BeautifulSoup(html, "html.parser").get_text()
    elif file_extension == '.txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")
    

    

In [5]:
raw_text = parse_file(file_path)

In [6]:
print(raw_text)

Project Title: Document Summarization using Retrieval-Augmented
Generation (RAG)
Objective:
To develop a summarization system that combines retrieval-based context
selection with large language model (LLM) generation. The system should
accept a long document and generate a concise, coherent summary using
semantic chunking and RAG.

Project Tasks:
1. Document Ingestion
● Accept documents in PDF, TXT, or Markdown format.
● Split into semantically meaningful chunks using sliding windows or
semantic segmenters.
2. Embedding & Retrieval
● Convert chunks to vector embeddings using SentenceTransformers or
OpenAI API. ● Store in FAISS or Chroma vector DB.
● Perform semantic retrieval for a general summary query (e.g.,
"Summarize this document").
3. Summary Generation
● Use top-k retrieved chunks and pass them into a pre-trained LLM
(e.g., GPT, LLaMA, Mistral).
● Generate a final summary that is coherent, fluent, and accurate.
4. Output Presentation
● Display the retrieved context and the gener

### Preprocessing

Cleaning the text

In [7]:

def clean_text(text):
    # Normalize line breaks and spaces
    text = re.sub(r'\r\n|\r', '\n', text)           # Convert \r\n or \r to \n
    text = re.sub(r'\n{2,}', '\n\n', text)          # Collapse many newlines into 2
    text = re.sub(r'[ \t]+', ' ', text)             # Remove extra spaces/tabs

    # Normalize unicode 
    text = unicodedata.normalize("NFKD", text)
    def add_period_to_bullet(match):
        line = match.group(0).strip()
        if not line.endswith('.'):
            return line + '.'
        return line

    # Add periods to lines that start with bullet markers (before removing markers)
    text = re.sub(r'(?m)^\s*[-*+]\s+(.*)', lambda m: "- " + add_period_to_bullet(m), text)
    # Remove common bullet points
    text = re.sub(
        r'[\u2022\u2023\u25E6\u2043\u2219\u25AA\u25AB\u25CB\u25CF\u25A0\u25B8\u29BE\u29BF]',
          '', text)

    # Remove markdown or ASCII-style tables
    text = re.sub(r'\|.*?\|', '', text)      # Remove markdown tables
    text = re.sub(r'[-=]{3,}', '', text)     # Remove underlines in tables
    text = re.sub(r'^\s*[\-\*+]\s+', '', text, flags=re.MULTILINE)  # Bulleted list lines

    # Remove figure/table/image captions
    text = re.sub(r'(Figure|Table|Image|Chart|Diagram)\s*\d+[\.:]?', '', text, flags=re.IGNORECASE)

    # Remove bracketed footnotes like [1], [12], (Fig. 3), etc.
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\(.*?fig.*?\)', '', text, flags=re.IGNORECASE)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Fix line breaks and hyphens split across lines
    text = re.sub(r'-\n', '', text)  # Remove hyphenated line-breaks
    text = re.sub(r'\n+', '\n', text)  # Collapse newlines
    text = re.sub(r'[ \t]+', ' ', text)  # Normalize spaces

    # Strip remaining non-ASCII or odd symbols
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # before every \n add a period if it doesn't end with one special character
    text = re.sub(r'(?<![.!?:])\n', '. \n', text)     

    return text.strip()


In [8]:
text = clean_text(raw_text)
print(text)

Project Title: Document Summarization using Retrieval-Augmented. 
Generation (RAG). 
Objective:
To develop a summarization system that combines retrieval-based context. 
selection with large language model (LLM) generation. The system should. 
accept a long document and generate a concise, coherent summary using. 
semantic chunking and RAG.
Project Tasks:
1. Document Ingestion. 
 Accept documents in PDF, TXT, or Markdown format.
 Split into semantically meaningful chunks using sliding windows or. 
semantic segmenters.
2. Embedding & Retrieval. 
 Convert chunks to vector embeddings using SentenceTransformers or. 
OpenAI API. Store in FAISS or Chroma vector DB.
 Perform semantic retrieval for a general summary query (e.g.,. 
"Summarize this document").
3. Summary Generation. 
 Use top-k retrieved chunks and pass them into a pre-trained LLM. 
(e.g., GPT, LLaMA, Mistral).
 Generate a final summary that is coherent, fluent, and accurate.
4. Output Presentation. 
 Display the retrieved conte

## Chunking

Download the embedding model from hugging face

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\fati1\AppData\Local\Temp\ipykernel_9724\1656931266.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
d:\RAGSummarizer\attempt1\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We will be using the semantic Chunker from langchain

In [10]:
semantic_chunker = SemanticChunker(embedding_model, breakpoint_threshold_type="percentile")


In [11]:
chunks = semantic_chunker.split_text(text)

metadatas = [
    {
        "source": file_path,
        "chunk_index": i,
        "length": len(chunk)

    }
    for i in range(len(chunks))
]


NameError: name 'chunk' is not defined

In [ ]:
print(f"Total number of chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} length: {len(chunk)} characters")
    print(f"Chunk {i+1}:\n{chunk}\n")

Total number of chunks: 6

Chunk 1 length: 1208 characters
Chunk 1:
Project Title: Document Summarization using Retrieval-Augmented. Generation (RAG). Objective:
To develop a summarization system that combines retrieval-based context. selection with large language model (LLM) generation. The system should. accept a long document and generate a concise, coherent summary using. semantic chunking and RAG. Project Tasks:
1. Document Ingestion. Accept documents in PDF, TXT, or Markdown format. Split into semantically meaningful chunks using sliding windows or. semantic segmenters. 2. Embedding & Retrieval. Convert chunks to vector embeddings using SentenceTransformers or. OpenAI API. Store in FAISS or Chroma vector DB. Perform semantic retrieval for a general summary query (e.g.,. "Summarize this document"). 3. Summary Generation. Use top-k retrieved chunks and pass them into a pre-trained LLM. (e.g., GPT, LLaMA, Mistral). Generate a final summary that is coherent, fluent, and accurate. 4. 

## Vector Storage

In [ ]:
db = Chroma(
    persist_directory="chroma_store",
    embedding_function=embedding_model
)

C:\Users\fati1\AppData\Local\Temp\ipykernel_21688\2075795383.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [ ]:
db.add_texts( texts=chunks, metadatas=metadatas)

['e43d4365-5997-4c86-a174-9f6c76d5ce64',
 '34621035-0ae9-495e-be6f-718cc4f5630d',
 '0a684e0c-e222-4092-bc55-63212e203e39',
 '3f8217eb-25bc-4cc3-aac6-71394330b725',
 '324191a0-0e34-41a4-90e9-0365d4888310',
 'a9b5d231-49dc-4dff-a428-39b89b26812f']

In [ ]:
db.persist()

C:\Users\fati1\AppData\Local\Temp\ipykernel_21688\123899826.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


## Query 